## Benchmarking NeuralNet.jl

### Load in required files for benchmarking

In [ ]:
include("src/algorithms/nn.jl");
include("src/utils/parse.jl");

using Gadfly;
using CategoricalArrays;
using DataFrames;

### First let's benchmark the autoencoder sum of squared error as a function of the learning rate and number of epochs.

In [ ]:
lr = [0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001];
epochs = [1000, 5000, 10000, 50000, 100000, 500000];
T = 5;

results = zeros(6, 6);

for alpha in 1:length(lr)
    for N in 1:length(epochs)
        println(string(lr[alpha], "\t", epochs[N]))
        res = 0.0;
        for i in 1:T
            sserr = encode_8x3x8(epochs[N], lr[alpha], ""; write_output=false, verbose=false)[4];
            res += sserr
        end
        results[N, alpha] = res / T;

    end
end

results

In [ ]:
resultdf = DataFrame(sserr = results[1,:], lr = lr, N=epochs[1])
resultdf = vcat(resultdf, DataFrame(sserr = results[2,:], lr=lr, N=epochs[2]))
resultdf = vcat(resultdf, DataFrame(sserr = results[3,:], lr=lr, N=epochs[3]))
resultdf = vcat(resultdf, DataFrame(sserr = results[4,:], lr=lr, N=epochs[4]))
resultdf = vcat(resultdf, DataFrame(sserr = results[5,:], lr=lr, N=epochs[5]))
resultdf = vcat(resultdf, DataFrame(sserr = results[6,:], lr=lr, N=epochs[6]))

plot(resultdf, y="sserr", x="lr", color="N", Geom.line, Guide.ylabel("Sum Sq. Error"), Guide.xlabel("Learning Rate"))

Using the optimal parameters we found in the previous section, let's look at the hidden layer's representation of the input.

In [ ]:
alpha = 0.05
N = 100000
h_out = encode_8x3x8(N, alpha, ""; write_output=false, verbose=false)[3];
h_out

## Now let's turn to the neural net where we will benchmark the size of the hidden layer first. 

In [ ]:
pos_seq_fp = "data/rap1-lieb-positives.txt";
total_seq_fp = "data/yeast-upstream-1k-negative.fa";
test_fp = "data/rap1-lieb-test.txt";

tdata, labels = parse_input(pos_seq_fp, total_seq_fp; balance=1);

In [ ]:
hl_sizes = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12];
T = 5
preds = zeros(length(hl_sizes))

for hl_size in 1:length(hl_sizes)
    pred = 0.0
    println(hl_sizes[hl_size]);
    for i in 1:T
        pred += nn_3layer(tdata, labels, hl_size, 1000, 0.05, "")[5]
    end
    preds[hl_size] = pred / T
end
    
plot(x=hl_sizes, y=preds, Guide.xlabel("Hidden Layer Size"), Guide.ylabel("Average Predictive Ability"))

### Now let's evaulate the effect of training set bias on our ability to train the network.

In [ ]:
biases = [1, 2, 3, 4, 5]
T = 5
preds = zeros(length(biases))

for bias in 1:length(biases)
    pred = 0.0
    println(biases[bias])
    for i in 1:T
        tdata, labels = parse_input(pos_seq_fp, total_seq_fp; balance=biases[bias]);
        pred += nn_3layer(tdata, labels, 11, 3000, 0.05, "")[5]
    end
    preds[bias] = pred / T
end

plot(x=biases, y=preds, Guide.xlabel("Bias Factor"), Guide.ylabel("Average Predictive Ability"), Geom.line)

### Finally, let's find the optimal set of learning parameters using cross-validation.

In [ ]:
Ns = [1000, 3000, 5000];
alphas = [0.05, 0.01, 0.005, 0.001];

results = zeros(3, 4);

for i in 1:length(Ns)
    for j in 1:length(alphas)
        println("$(Ns[i]), $(alphas[j])")
        fpred = train_nn(tdata, labels, 11, Ns[i], alphas[j], ""; write_output = false, verbose = false, C=10);
        results[i, j] = fpred
    end
end

results

In [ ]:
resultdf = DataFrame(pred = results[:,1], lr = alphas[1], N=Ns)
resultdf = vcat(resultdf, DataFrame(pred = results[:,2], lr=alphas[2], N=Ns))
resultdf = vcat(resultdf, DataFrame(pred = results[:,3], lr=alphas[3], N=Ns))
resultdf = vcat(resultdf, DataFrame(pred = results[:,4], lr=alphas[4], N=Ns))


plot(resultdf, y="pred", x="N", color="lr", Geom.line, Guide.ylabel("Average Predictive Value"), Guide.xlabel("Number of Epochs"),
    Coord.cartesian(xmin=1000, xmax=5000), Guide.colorkey(title="Learning Rate"))

### From the plot above, it's apparent that  the best combination of parameters is around $N = 5000$ and $\alpha = 0.05$. Now let's look at how sensitive these parameters are to class inbalance.

In [ ]:
biases = [1, 2, 3, 4, 5]
preds = zeros(length(biases))

N = 5000
alpha = 0.05

for bias in 1:length(biases)
    println(biases[bias])
    tdata, labels = parse_input(pos_seq_fp, total_seq_fp; balance=biases[bias]);
    preds[bias] = train_nn(tdata, labels, 11, N, alpha, ""; write_output = false, verbose = false, C=10);
end

plot(x=biases, y=preds, Guide.xlabel("Bias Factor"), Guide.ylabel("Average Predictive Ability"), Geom.line)

## Now that we've benchmarked and selected parameters, we can predict on the test data set...

In [ ]:
tdata, labels = parse_training(pos_seq_fp, total_seq_fp; balance=5);
test_data = parse_testing(test_fp)

final_accur = nn_predict_on_data(tdata, labels, test_data, 11, 5000, 0.05, "final_predictions.txt"; C = 20);
